In [11]:
import json, requests, os, re, datetime

basic_land_types = ['Plains', 'Island', 'Swamp', 'Mountain', 'Forest']
     
def getValidFilename(s): #makes a "good" filename to save the file (removes spaces and slashes, etc).
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)


def checkdir(dir_path): #checks if the directory for the set already exists. If not it creates it.
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)

        
def getSetCode(s): #Another option to label sets by their 3 letter designation wrapped in parenthesis i.e., (M19)
    return "(" + str(s).upper() + ")"


def writefile(url, file_path): #checks if the file already exists. If not it downloads the image and creates the file.
    if not os.path.isfile(file_path):
        r = requests.get(url)
        open(file_path, 'wb').write(r.content)


def getUniqueArtData(): #downloads the current unique-artwork.json file from scryfall. Just reads to memory, doesn't save it to a file.
                       #this part takes a while. If you're running the script and don't see any files being written for a bit, wait longer.
    request_url = requests.get('https://api.scryfall.com/bulk-data')
    request_data = requests.get(request_url.json()['data'][1]['download_uri'])
    return request_data.json()
 
start=datetime.datetime.now()
print("Writing files to " + os.path.join(os.getcwd(), "art"))
 
print("Starting at: " + str(start))
 
for card in getUniqueArtData(): #reads each card in the all-cards file
    if card['lang'] == 'en' and card['set_type'] in ['core', 'expansion', 'draft_innovation'] and card['name'] not in basic_land_types: 
            #checks if it's an english card from a standard set (or supplementary) and not a basic land.
            #see all set types: https://scryfall.com/docs/api/sets
        dir_path = "art\\" + getValidFilename(card['set_name']) 
        #+ getSetCode(card['set'])
        checkdir(dir_path) #creates the set directory if it doesn't already exist
        if 'image_uris' in card: #if it's a regular card, writes the file
            file_path = dir_path + "\\" + getValidFilename(card['name']) + '.png'
            writefile(card['image_uris']['border_crop'],file_path)
        else: #if it's a twosided card, writes both sides
            if card['type_line'] != 'Card // Card': 
                    #if the type is not "Card // Card" (scryfall has a weird cases where both sides are the same that act weird, ignoring these)
                file_path = dir_path + "\\" + getValidFilename(card['card_faces'][0]['name']) + '_front.png'
                writefile(card['card_faces'][0]['image_uris']['border_crop'],file_path)
                file_path = dir_path + "\\" + getValidFilename(card['card_faces'][0]['name']) + '_rear.png'
                writefile(card['card_faces'][1]['image_uris']['border_crop'],file_path)
                    
end=datetime.datetime.now()
print("Finished at " + str(end) + ". Elapsed time: " + str(end-start))

Writing files to E:\Users\Jarod\Stuff\Project Stuff\Magic Project\art
Starting at: 2023-08-09 10:29:41.720504
Finished at 2023-08-09 11:58:26.528155. Elapsed time: 1:28:44.807651
